**PDF File Questioning & Answering - LlamaIndex, Llama2**

---

---



seamless interaction with PDF files, Chat with your PDF files using LlamaIndex, DataStax Astra DB (Apache Cassandra), and Gradient's open-source models, including LLama2


---


---
**Llama2**: Open source large language model

---


**LlamaIndex**: Data framework for large language model applications

---

**Vector Database**: DataStax implementation of Apache Cassandra


---

**Gradient's LLM** ais used to access the Llama2

---




In [ ]:
#Installing the Libraries
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.7/169.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.3/929.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

In [ ]:
#Importing the JSON Module and OS
import os
import json
from google.colab import userdata
#Inorder to use the Gradient LLM's we need the access tokens
#Access Tokens and Workspace ID has to be retrived from Gradient workspace https://auth.gradient.ai/select-workspace
os.environ['GRADIENT_ACCESS_TOKEN'] = userdata.get('GRADIENT_ACCESS_TOKEN')
os.environ['GRADIENT_WORKSPACE_ID'] =  userdata.get('GRADIENT_WORKSPACE_ID')
#The Tokens and work ID's has been stored in the Google colab Secrets(at the left side of the colab)

In [ ]:
#Importing Llama indeex and Cassandra
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings import GradientEmbedding
from llama_index.llms import GradientBaseModelLLM
from llama_index.vector_stores import CassandraVectorStore
import cassandra
print (cassandra.__version__) #Checking the version of Cassandra has been imported

3.28.0


In [ ]:
#To store the indexes we need Vector Database
#Connecting to Vector Database - Datastax Astra DB
#The below code is from DataStax Astra DB official Website. You can access to it after creating the Database in DataStax
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# This secure connect bundle is autogenerated when you download your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-fileqa-db.zip'  #Using Secure Connection Bundles when connecting to the database with drivers
}      #This bundle can be downloaded, after creating the vector database while setting up the connection.

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("/content/fileqa_db-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(138560424797488) aeadb65c-8c30-4665-ade6-686d29706503-us-east1.db.astra.datastax.com:29042:f88d0cb1-9d05-413f-bbb7-400aafa21446> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.11-8a763e30acd8


In [ ]:
#Setting up the Environment API Key
import os
from getpass import getpass

if not os.environ.get("GRADIENT_ACCESS_TOKEN", None):
    # Access token under https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_ACCESS_TOKEN"] = getpass("gradient.ai access token:")
if not os.environ.get("GRADIENT_WORKSPACE_ID", None):
    # `ID` listed in `$ gradient workspace list`
    # also displayed after login at at https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_WORKSPACE_ID"] = getpass("gradient.ai workspace id:")

In [ ]:
#Define the Gradient's Model Adapter for LLAMA-2

from llama_index.llms import GradientBaseModelLLM

# You can also use a model adapter you've trained with GradientModelAdapterLLM
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

In [20]:
#Configuring the Gradient embeddings
#Embedding is used in order to generate the index
embed_model = GradientEmbedding(             #Gradient Embedding function
    gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)
#Executing the LlamaIndex
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=1024,
)

set_global_service_context(service_context)
print(service_context)

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=1024, num_output=400, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=GradientEmbedding(model_name='bge-large', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7e051a2a7af0>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7e051a2a7af0>, chunk_size=256, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7e051de070d0>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7e051a2a7af0>)


In [24]:
documents = SimpleDirectoryReader("/content/Documents").load_data()
print(f"Loaded {len(documents)} document(s).")


Loaded 101 document(s).


In [ ]:
index = VectorStoreIndex.from_documents(
    documents,service_context=service_context)
query_engine = index.as_query_engine()

# **Shoot the question you wanted!**

In [19]:
response = query_engine.query("function of sql")
print(response)


There are several functions in SQL that can be used to perform calculations on data. Some of the most common functions include:

1. AVG(): Returns the average value of a column.
2. COUNT(): Returns the number of rows in a table.
3. FIRST(): Returns the first value in a column.
4. LAST(): Returns the last value in a column.
5. MAX(): Returns the largest value in a column.
6. MIN(): Returns the smallest value in a column.
7. SUM(): Returns the sum of a numeric column.

Additionally, there are scalar functions in SQL that can be used to perform calculations on a single value. Some of the most common scalar functions include:

1. UCASE(): Converts a field to upper case.
2. LCASE(): Converts a field to lower case.

It is important to use aggregate functions and scalar functions appropriately in order to perform the desired calculations on data.
